# <span style="color:purple;"> Ai virtual keyboard</span>



In [ ]:
pip install pyautogui


### <span style="color:orange;">import neccecssary libraries 📚</span>


In [1]:

import cv2
import mediapipe as mp
import pyautogui
import numpy as np
import math


### <span style="color:orange;">hands detection ✋🤚</span>


In [2]:
# Initialize MediaPipe hand detection
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

### <span style="color:orange;">Define and draw the virtual keyboard ⌨️ 🖥️ </span>


In [3]:
keyboard_keys = [
    ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
    ["A", "S", "D", "F", "G", "H", "J", "K", "L"],
    ["Z", "X", "C", "V", "B", "N", "M"],
    ["Space"]
]

key_size = 80
start_x, start_y = 50, 100
typed_text = ""
finger_touching = False  # Track whether fingers are touching

In [4]:
# Function to draw the virtual keyboard on the frame
def draw_keyboard(img):
    global start_x, start_y
    y = start_y
    for row in keyboard_keys:
        x = start_x
        for key in row:
            # If it's the spacebar, draw it larger
            if key == "Space":
                cv2.rectangle(img, (x, y), (x + key_size * 6 + 50, y + key_size), (128, 0, 128), 3)  # Spacebar spans 6 keys
                cv2.putText(img, key, (x + 150, y + 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)  # Larger text for spacebar
            else:
                cv2.rectangle(img, (x, y), (x + key_size, y + key_size), (128, 0, 128), 3)
                cv2.putText(img, key, (x + 20, y + 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            x += key_size + 10
        y += key_size + 10


In [5]:
# Function to calculate the distance between two points
def calculate_distance(point1, point2):
    return math.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)


### <span style="color:orange;">check key pressed functions and reset touch ⌨️✅🔑</span>


In [11]:
# Function to check if the thumb and index finger are touching
def check_key_press(landmarks):
    global typed_text, finger_touching
    y = start_y
    for row in keyboard_keys:
        x = start_x
        for key in row:
            if landmarks:
                # Get the tip of the thumb and index finger
                thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP]
                index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                
                # Calculate the distance between the thumb and index finger
                distance = calculate_distance(
                    (thumb_tip.x * frame.shape[1], thumb_tip.y * frame.shape[0]),
                    (index_tip.x * frame.shape[1], index_tip.y * frame.shape[0])
                )
                
                # If the distance is below a certain threshold (fingers are touching)
                if distance < 30:  # Adjust this threshold as needed
                    # Check if we haven't already pressed the button
                    if not finger_touching:
                        # Check if we are touching the spacebar
                        if key == "Space" and x < index_tip.x * frame.shape[1] < x + key_size * 6 + 50 and y < index_tip.y * frame.shape[0] < y + key_size:
                            typed_text += " "  # Add a space when spacebar is pressed
                            pyautogui.write(" ")  # Simulate typing the space
                            finger_touching = True  # Mark that we have already pressed
                            return  # Exit after the key is pressed
                        # Check for other keys
                        elif key != "Space" and x < index_tip.x * frame.shape[1] < x + key_size and y < index_tip.y * frame.shape[0] < y + key_size:
                            typed_text += key
                            pyautogui.write(key)  # Simulate typing the key
                            finger_touching = True  # Mark that we have already pressed
                            return  # Exit after the key is pressed
            x += key_size + 10
        y += key_size + 10


In [12]:
# Function to reset the touch state when fingers are not touching
def reset_touch_state(landmarks):
    global finger_touching
    thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    
    # Calculate the distance between thumb and index finger
    distance = calculate_distance(
        (thumb_tip.x * frame.shape[1], thumb_tip.y * frame.shape[0]),
        (index_tip.x * frame.shape[1], index_tip.y * frame.shape[0])
    )
    
    # If the distance is larger than the threshold, fingers are not touching, so reset the state
    if distance > 30:
        finger_touching = False





# <span style="color:teal;">Execution" 🏃‍♂️💻⚡</span>


In [15]:
# Start the video capture 
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to make it larger
    frame_width = 1028  # Set desired width
    frame_height = 720  # Set desired height
    frame = cv2.resize(frame, (frame_width, frame_height))  # Resize the frame

    # Flip the frame horizontally for a mirror view
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    result = hands.process(rgb_frame)

    # Draw the virtual keyboard
    draw_keyboard(frame)

    # If hands are detected, process the landmarks
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks on the frame
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Extract landmarks (positions of the hand points)
            landmarks = [landmark for landmark in hand_landmarks.landmark]
            
            # Check for button press when thumb and index finger touch
            check_key_press(landmarks)
            
            # Reset touch state when fingers are not touching
            reset_touch_state(landmarks)

    # Display the typed text on the screen with padding below the keyboard
    cv2.putText(frame, typed_text, (50, frame.shape[0] - 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 5)

    # Show the frame with the virtual keyboard and hand landmarks
    cv2.imshow("Virtual Keyboard", frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()
typed_text = ""
